In [1]:
import pandas as pd
import numpy as np

In [2]:
# look at our SNP allele frequency
# .afreq file obtained from running:
#./plink2 --bfile gene_data/ukb22418_cY_b0_v2 --freq --out gene_data/chrY_snp
snps = pd.read_csv("gene_data/chrY_snp.afreq",sep="\t")
print(snps.shape)
snps.head()

(691, 6)


,#CHROM,ID,REF,ALT,ALT_FREQS,OBS_CT
0,Y,rs1800865,A,G,1.000000,24933
1,Y,Affx-89009265,G,T,0.999996,223019
2,Y,Affx-89005343,G,A,0.999996,223219
3,Y,Affx-89024591,A,G,0.999973,223303
4,Y,Affx-89014410,A,G,0.999987,222556


In [3]:
# get those just with rsids (common terminology for SNP IDs)
rsids = [x for x in snps["ID"] if x[:2]=="rs"]
rsid_snps = snps.loc[snps["ID"].isin(rsids)].copy()

rsid_snps["REF_FREQS"] = 1 - rsid_snps["ALT_FREQS"]

rsid_snps.shape

(162, 7)

In [4]:
rsid_snps["ID"].to_csv("chrY_rsids.csv",index=False)

In [5]:
## ran the RSIDs through the ENSEMBL variant effect predictor @ http://grch37.ensembl.org/Homo_sapiens/Tools/VEP
## downloaded as a .txt locally and converted to a .csv to upload

vep = pd.read_csv("gene_data/chrY_snps_VEP.csv")
vep.head()

,#Uploaded_variation,Location,Allele,Consequence,IMPACT,SYMBOL,Gene,Feature_type,Feature,BIOTYPE,...,PHENO,PUBMED,MOTIF_NAME,MOTIF_POS,HIGH_INF_POS,MOTIF_SCORE_CHANGE,TRANSCRIPTION_FACTORS,ada_score,rf_score,BLOSUM62
0,rs1800865,Y:2658271-2658271,A,upstream_gene_variant,MODIFIER,SRY,ENSG00000184895,Transcript,ENST00000383070.1,protein_coding,...,-,-,-,-,-,-,-,-,-,-
1,rs1800865,Y:2658271-2658271,A,non_coding_transcript_exon_variant,MODIFIER,RNASEH2CP1,ENSG00000237659,Transcript,ENST00000454281.1,processed_pseudogene,...,-,-,-,-,-,-,-,-,-,-
2,rs1800865,Y:2658271-2658271,A,upstream_gene_variant,MODIFIER,SRY,ENSG00000184895,Transcript,ENST00000525526.2,protein_coding,...,-,-,-,-,-,-,-,-,-,-
3,rs1800865,Y:2658271-2658271,A,upstream_gene_variant,MODIFIER,SRY,ENSG00000184895,Transcript,ENST00000534739.2,protein_coding,...,-,-,-,-,-,-,-,-,-,-
4,rs35284970,Y:2734854-2734854,T,synonymous_variant,LOW,RPS4Y1,ENSG00000129824,Transcript,ENST00000250784.8,protein_coding,...,-,28315050,-,-,-,-,-,-,-,-


In [6]:
vepsnps = pd.merge(rsid_snps,vep,how="right",left_on="ID",right_on="#Uploaded_variation")
vepsnps.shape

(625, 51)

In [17]:
imp_snps = vepsnps.loc[vepsnps["Consequence"].isin(["3_prime_UTR_variant","downstream_gene_variant",
                                                        "intron_variant","missense_variant",
                                                        "regulatory_region_variant",
                                                        "splice_region_variant,intron_variant",
                                                        "upstream_gene_variant"])].copy()
imp_snps.shape

(385, 51)

In [18]:
imp_snps["START"] = [x.split("Y:")[1].split("-")[0] for x in imp_snps["Location"]]
yq_snps = imp_snps.loc[imp_snps["START"].astype(int) >= 12000000].copy()
yq_snps.shape

(352, 52)

In [19]:
len(np.unique(yq_snps["#Uploaded_variation"]))

78

In [20]:
common_snps = yq_snps.loc[(yq_snps["REF_FREQS"]>=0.01)&(yq_snps["ALT_FREQS"]>=0.01)].copy()
common_snps = common_snps.loc[common_snps["SYMBOL"]!="-"]
common_snps.shape

(145, 52)

In [21]:
len(np.unique(common_snps["#Uploaded_variation"]))

31

In [22]:
interesting_snps = common_snps[["ID","SYMBOL","ALT_FREQS","Consequence"]].drop_duplicates()
interesting_snps.to_csv("gene_data/chrY_interesting_snps.csv")
interesting_snps.shape

(43, 4)

In [23]:
interesting_snps.SYMBOL.value_counts()

DDX3Y      11
USP9Y      10
EIF1AY      6
UTY         5
NLGN4Y      3
RCC2P1      2
MXRA5P1     2
KDM5D       2
RPS4Y2      1
ARSEP1      1
Name: SYMBOL, dtype: int64

In [27]:
np.unique(interesting_snps.sort_values(by="SYMBOL").ID)

array(['rs112386595', 'rs112910996', 'rs113875126', 'rs113899240',
       'rs13305774', 'rs13447352', 'rs13447354', 'rs1558843', 'rs16980548',
       'rs17221964', 'rs17222244', 'rs17222279', 'rs17307070',
       'rs17307294', 'rs2020857', 'rs2032598', 'rs2032604', 'rs2032624',
       'rs2032631', 'rs2032658', 'rs2713254', 'rs34630819', 'rs35108305',
       'rs7067269', 'rs72625379', 'rs76415739', 'rs7893052', 'rs9341296',
       'rs9341301', 'rs9786043', 'rs9786153'], dtype=object)

In [39]:
ids = pd.read_csv("clinical_data/cohort_ids.csv",header=None)
ids["fam"]=ids
ids.to_csv("clinical_data/cohort_ids.txt",sep="\t",index=False,header=False)

In [43]:
!./plink --bfile gene_data/ukb22418_cY_b0_v2 --recode AD --keep clinical_data/cohort_ids.txt --out gene_data/chrY

PLINK v1.90b6.21 64-bit (19 Oct 2020)          www.cog-genomics.org/plink/1.9/
(C) 2005-2020 Shaun Purcell, Christopher Chang   GNU General Public License v3
Logging to gene_data/chrY.log.
Options in effect:
  --bfile gene_data/ukb22418_cY_b0_v2
  --keep clinical_data/cohort_ids.txt
  --out gene_data/chrY
  --recode AD

128678 MB RAM detected; reserving 64339 MB for main workspace.
691 variants loaded from .bim file.
488377 people (223459 males, 264780 females, 138 ambiguous) loaded from .fam.
Ambiguous sex IDs written to gene_data/chrY.nosex .
--keep: 181978 people remaining.
Using 1 thread (no multithreaded calculations invoked).
Before main variant filters, 181978 founders and 0 nonfounders present.
Calculating allele frequencies... 10111213141516171819202122232425262728293031323334353637383940414243444546474849505152535455565758596061626364656667686970717273747576777879808182838485868788899091929394959697989 done.
Total genotyping rate in remaining samples is 0.856194.
691 variants

In [57]:
# snp_df = pd.read_csv("gene_data/chrY.raw",sep=" ")
cols = [x for x in snp_df.columns if x.split("_")[0] in ["rs2032598","rs2032631"]]
cols = ["FID"]+cols

In [68]:
snp_df[cols].head()

,FID,rs2032598_C,rs2032598_HET,rs2032631_G,rs2032631_HET
0,2945717,0.0,0.0,2.0,0.0
1,3639347,0.0,0.0,2.0,0.0
2,4897831,0.0,0.0,2.0,0.0
3,3016210,0.0,0.0,0.0,0.0
4,4045582,0.0,0.0,2.0,0.0


In [75]:
snp_df = snp_df[["FID","rs2032598_C","rs2032631_G"]]
snp_df2 = snp_df[["FID","rs2032598_C"]]
snp_df2.columns = ["userId","exposure"]
snp_df2.dropna().to_csv("gene_data/rs2032598_C.csv",index=False)
snp_df = snp_df[["FID","rs2032598_C","rs2032631_G"]]

snp_df2 = snp_df[["FID","rs2032631_G"]]
snp_df2.columns = ["userId","exposure"]
snp_df2.dropna().to_csv("gene_data/rs2032631_G.csv",index=False)